In [1]:
import numpy as np
import queue
import copy
import matplotlib.pyplot as plt


In [3]:
# unit: hour
#即核酸检测有两个小时的检测窗口 eg. 9-11 a.m. 
#假设所有人都在这个区间进行检测👉后续可以优化
# simulation(hours)
#arrival rate (/hour) i.e. 2 per min
#service rate (/hour) 这里考虑改进前 即50s左右
#我们假设在正式检测开始之前，已经有人开始排队👉即这里的‘begin', 是一个0-30之间的随机整数
#number of server

begin = np.random.randint(0,30)
c = 1
total_time = 2
IAT_rate = 120
ST_rate = 72*c
rho = IAT_rate/ST_rate
print(begin)

11


In [4]:
# Initialize Parameters
# IAT  empty array for putting random number inter-arrival time
# St   ... for serving time rate
# AT: arrival times
# ST: serve time


qu = queue.Queue()
curr_process = None
IAT = []
ST = []
AT = []
wait_time = []
server_busy = False
list_wait = []
list_delay = []


In [5]:
# num_processes 即一共有多少个人在检测窗口里来做核酸检测

num_processes = int(np.random.poisson(IAT_rate)* total_time) 
total_people = num_processes + begin
num_processes_served = 0
print(num_processes)
print(begin)
print(total_people)

244
11
255


In [6]:
IAT_new = []

In [7]:
#Populate new Inter-Arrival-Times (IAT_new)
#即在正式检测开始之前到达的人的inter arrival time=0

for i in range(total_people):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i<begin:
        IAT_new.append(0)
        i=i+1
    else: 
        IAT_new.append(int(temp-temp%1))

print(IAT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 85, 22, 2, 82, 36, 8, 20, 15, 6, 9, 0, 16, 34, 53, 156, 27, 52, 5, 7, 92, 55, 1, 31, 7, 84, 29, 53, 16, 6, 2, 28, 20, 19, 15, 68, 10, 6, 8, 24, 36, 11, 50, 37, 29, 20, 6, 12, 36, 73, 60, 17, 33, 39, 21, 27, 22, 46, 11, 48, 12, 26, 27, 45, 90, 46, 10, 12, 6, 2, 11, 85, 62, 13, 34, 13, 17, 52, 8, 28, 66, 71, 23, 37, 181, 18, 111, 73, 20, 28, 39, 25, 0, 0, 24, 15, 42, 5, 2, 16, 0, 13, 0, 38, 37, 43, 31, 8, 12, 7, 8, 27, 17, 26, 1, 5, 82, 5, 41, 34, 12, 4, 26, 22, 13, 35, 9, 1, 7, 124, 0, 16, 58, 19, 20, 47, 16, 6, 58, 0, 5, 14, 3, 5, 6, 10, 67, 12, 14, 8, 41, 8, 34, 16, 3, 61, 30, 13, 5, 14, 54, 101, 49, 45, 14, 58, 36, 36, 78, 11, 17, 9, 17, 3, 3, 1, 131, 16, 37, 63, 34, 73, 3, 27, 61, 22, 34, 19, 98, 49, 8, 24, 9, 3, 9, 58, 33, 62, 48, 131, 18, 92, 82, 47, 33, 96, 49, 32, 12, 73, 20, 38, 109, 3, 57, 24, 8, 53, 6, 25, 8, 1, 39, 4, 4, 49, 125, 46, 0, 31, 3, 4, 40, 15, 15, 5, 56, 10, 11, 29, 16, 4, 7, 9, 20]


In [8]:
# Populate Service-Times (ST) (where ST[i]!=0)

while not len(ST) == total_people:
    temp = np.random.exponential(1/ST_rate)*60*60
    if not int(temp- temp%1)<1:
        ST.append(int(temp - temp%1))

print(ST)

[6, 9, 72, 21, 16, 52, 50, 184, 10, 45, 1, 5, 83, 138, 4, 167, 15, 5, 20, 20, 87, 21, 14, 40, 2, 13, 5, 54, 88, 37, 30, 19, 47, 4, 64, 28, 100, 9, 37, 3, 3, 18, 192, 9, 57, 98, 97, 45, 129, 7, 23, 104, 166, 83, 108, 204, 77, 137, 82, 91, 68, 29, 85, 54, 12, 72, 33, 19, 39, 26, 53, 23, 27, 13, 64, 12, 138, 73, 50, 180, 21, 5, 8, 110, 23, 30, 65, 8, 138, 24, 40, 194, 19, 89, 50, 14, 10, 26, 114, 117, 162, 205, 84, 14, 27, 16, 34, 1, 19, 117, 12, 47, 197, 12, 49, 12, 6, 7, 106, 51, 178, 116, 49, 57, 8, 8, 13, 143, 3, 28, 15, 12, 7, 78, 28, 26, 49, 4, 129, 23, 3, 80, 22, 29, 31, 45, 157, 6, 16, 106, 59, 36, 85, 127, 17, 66, 66, 31, 18, 58, 21, 8, 56, 99, 26, 14, 5, 20, 23, 10, 23, 29, 58, 84, 47, 60, 3, 52, 73, 59, 26, 18, 34, 24, 65, 10, 6, 174, 100, 53, 180, 25, 152, 16, 3, 76, 59, 104, 65, 12, 42, 6, 113, 15, 63, 85, 39, 35, 123, 13, 49, 24, 44, 22, 60, 230, 154, 31, 42, 10, 36, 9, 24, 19, 8, 26, 84, 6, 173, 55, 43, 68, 29, 4, 57, 76, 115, 72, 65, 13, 52, 82, 54, 11, 72, 46, 87, 56, 70,

In [9]:
# save a copy of ST

ST_copy = copy.deepcopy(ST)

In [10]:
AT_new = []

In [11]:
# Get new Arrival-Times (AT) from IAT starting at t=0 
# 生成到达时间队列
# and initialize Waiting-Times to 0

for i in range(total_people):
    if i<begin:
        AT_new.append(0)
        i=i+1
    else:
        AT_new.append(IAT_new[i]+AT_new[i-1])
    wait_time.append(0)

print(AT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 85, 107, 109, 191, 227, 235, 255, 270, 276, 285, 285, 301, 335, 388, 544, 571, 623, 628, 635, 727, 782, 783, 814, 821, 905, 934, 987, 1003, 1009, 1011, 1039, 1059, 1078, 1093, 1161, 1171, 1177, 1185, 1209, 1245, 1256, 1306, 1343, 1372, 1392, 1398, 1410, 1446, 1519, 1579, 1596, 1629, 1668, 1689, 1716, 1738, 1784, 1795, 1843, 1855, 1881, 1908, 1953, 2043, 2089, 2099, 2111, 2117, 2119, 2130, 2215, 2277, 2290, 2324, 2337, 2354, 2406, 2414, 2442, 2508, 2579, 2602, 2639, 2820, 2838, 2949, 3022, 3042, 3070, 3109, 3134, 3134, 3134, 3158, 3173, 3215, 3220, 3222, 3238, 3238, 3251, 3251, 3289, 3326, 3369, 3400, 3408, 3420, 3427, 3435, 3462, 3479, 3505, 3506, 3511, 3593, 3598, 3639, 3673, 3685, 3689, 3715, 3737, 3750, 3785, 3794, 3795, 3802, 3926, 3926, 3942, 4000, 4019, 4039, 4086, 4102, 4108, 4166, 4166, 4171, 4185, 4188, 4193, 4199, 4209, 4276, 4288, 4302, 4310, 4351, 4359, 4393, 4409, 4412, 4473, 4503, 4516, 4521, 4535, 4589, 4690, 4739, 4784, 4798, 4856, 4892

In [12]:
# simulate M/M/1 queue

for i in range(total_time*60*60):    
    if server_busy:
        for item in list(qu.queue):
            wait_time[item] = wait_time[item] + 1
        ST[curr_process] = ST[curr_process] - 1
        if ST[curr_process] == 0:
            server_busy = False
            num_processes_served = num_processes_served + 1

    for j in range(total_people):
        if i== AT_new[j]:
            qu.put(j)

    if not server_busy and not qu.empty():
        curr_process = qu.get()
        server_busy = True
        


In [13]:
# 总共等待时间
# 总共delay时间 = total time in the system 
# delay = waiting time + serve time
sum_wait = 0
sum_delay = 0

In [14]:
for i in range(num_processes_served):
    sum_wait = sum_wait + wait_time[i]
    sum_delay = sum_delay + wait_time[i] + ST_copy[i]
    
if num_processes_served == 0:
    list_wait.append(0)
    list_delay.append(0)
else:
    list_wait.append(sum_wait/(num_processes_served*60*60))  
    list_delay.append(sum_delay/(num_processes_served*60*60))


In [16]:
print(total_people)
print(num_processes)
print(num_processes_served)

print(num_processes_served/total_people)

255
244
127
0.4980392156862745


In [17]:
#平均等待时间 in hour
print(sum_wait/(num_processes_served*60*60))
print(sum_delay/(num_processes_served*60*60))

# 平均等待时间 in minute
print(sum_wait/(num_processes_served*60))
print(sum_delay/(num_processes_served*60))

0.4548534558180227
0.4704636920384952
27.291207349081365
28.22782152230971


In [23]:
print((6000/worktime)*2)
print(salary/worktime)

65.21739130434783
50.14673913043478
